1) Problem Statement,Dataset Description,Inspection 
2) Initial Observation,Numeric cleaning
3) Date cleaning
4) Quality Checks & Validation
5) Final Clean Dataset,Key Learning

1) PROBLEM STATEMENT

Real world dataset often fail silently due to dirty numeric values, ambiguous dates, and inconsistent categorical entries.
The project focus on building a reliable data cleaning pipeline from first principles

2) Requirements  - Numeric mess,Date mess, Categorical drift

In [1]:
import pandas as pd
df=pd.read_csv(r"D:\ANALYST\DATASETs\Data Science\Dataset in Csv\RawdataProject.csv")
df.shape
df.info()
df.isna().sum()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order_ID      20 non-null     object
 1   Order_Date    19 non-null     object
 2   Customer_ID   20 non-null     object
 3   Product       20 non-null     object
 4   Category      20 non-null     object
 5   Unit_Price    19 non-null     object
 6   Quantity      20 non-null     int64 
 7   Total_Amount  19 non-null     object
 8   Discount      10 non-null     object
 9   Notes         19 non-null     object
 10  Unnamed: 10   1 non-null      object
dtypes: int64(1), object(10)
memory usage: 1.8+ KB


,Order_ID,Order_Date,Customer_ID,Product,Category,Unit_Price,Quantity,Total_Amount,Discount,Notes,Unnamed: 10
0,A001,2/3/2025,C101,Mobile,Electronics,₹12,1,12000,10%,Delivered,NaN
1,A002,3/4/2025,C102,laptop,Electronics,850 USD,1,850,NaN,urgent,NaN
2,A003,April 5th 2025,C103,Head phone,Audio,45$,2,90,NaN,Delayed,NaN
3,A004,4/5/2025,C104,Notebook,Electronics,1,0,"1,100",5%,NaN,-
4,A005,6/5/2025,C105,Mobile phone,Electronics,NaN,1,NaN,0,Cancelled,NaN
5,A006,5/7/2025,C106,mobile,electronics,infinite,1,infinite,NaN,Review,NaN
6,A007,8/1/2025,C107,LAPTOP,Electronics,780,2,1560,10%,Delivered late,NaN
7,A008,1-Aug-25,C108,Cellphone,Electronics,₹15,500,1,15500,OK,NaN
8,A009,8/1/2025,C109,Headphones,Audio,~3000,1,3000,?,Approx value,NaN
9,A010,9th Aug 2025,C110,Phone,Electronics,12000 INR,1,12000,5%,-,NaN


<!-- most messiest column -->

In [2]:
df['Unit_raw']=df['Unit_Price']
df['Unit_clean']=df['Unit_raw'].astype(str).str.strip()
df['Unit_clean'].value_counts(dropna=False)

Unit_clean
4500         2
850 USD      1
₹12          1
45$          1
1            1
infinite     1
nan          1
₹15          1
~3000        1
12000 INR    1
780          1
999.99       1
13k          1
1.20E+04     1
-12000       1
7000         1
₹ 11         1
850usd       1
12000±200    1
Name: count, dtype: int64

Semantic

In [3]:
df['unitmissing']=df['Unit_raw'].str.match(r'^\s*(n?a|nan|missing|none|N/A|'')\s*$')
df['is_infinte']=df['Unit_raw'].str.contains(r'\binf(inite)|saturation|overflow|ilux|ine\b',case=False,regex=True,na=False)
df['isrange']=df['Unit_raw'].str.contains(r'<|>|<=|>=|\bless than\b|\bgreater than\b|\bbelow\b',na=False)
df['approximate']=df['Unit_raw'].str.contains(r'\b(~|approx|about|approximately|around|nearly|almost)\b',case=False,na=False,regex=True)
df['uncertainity']=df['Unit_raw'].str.contains(r'±|\+-/|\+-','',na=False)
df['parenthesis']=df['Unit_raw'].str.contains(r'\(\s*-?\d',case=False,na=False,regex=True)
safemasking=~(df['unitmissing']|df['is_infinte']|df['isrange']|df['approximate']|df['parenthesis']|df['uncertainity'])
df.loc[safemasking,'Unit_raw']

C:\Users\ayush\AppData\Local\Temp\ipykernel_14320\3244429908.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['is_infinte']=df['Unit_raw'].str.contains(r'\binf(inite)|saturation|overflow|ilux|ine\b',case=False,regex=True,na=False)
C:\Users\ayush\AppData\Local\Temp\ipykernel_14320\3244429908.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['approximate']=df['Unit_raw'].str.contains(r'\b(~|approx|about|approximately|around|nearly|almost)\b',case=False,na=False,regex=True)


0           ₹12
1       850 USD
2           45$
3             1
4           NaN
6           780
7           ₹15
8         ~3000
9     12000 INR
10       999.99
11          13k
12         4500
13     1.20E+04
14       -12000
15         7000
16         ₹ 11
17       850usd
19         4500
Name: Unit_raw, dtype: object

In [72]:
df.loc[safemasking,'Unit_clean']=df.loc[safemasking,'Unit_clean'].astype(str).str.strip()
df.loc[safemasking,'Unit_clean']=df.loc[safemasking,'Unit_clean'].str.replace(r'[$₹]','',regex=True)
df.loc[safemasking,'Unit_clean']=df.loc[safemasking,'Unit_clean'].str.replace(r',','',regex=False)
df.loc[safemasking,'Unit_clean']=df.loc[safemasking,'Unit_clean'].str.lower()
df.loc[safemasking,'Unit_clean']=df.loc[safemasking,'Unit_clean'].str.replace(r'\b(usd|inr|INR|USD|RUPEES|rupees|euro|EURO)\b','',regex=True)
def handle_k (x):
    if x is None:
        return None
    x=str(x).strip()
    if x.lower().endswith('k'):
        number_part=x[:-1]
        return float(number_part)*1000
    return x
df['Unit_clean']=df['Unit_clean'].apply(handle_k)
df.loc[safemasking,'Unit_clean']=df.loc[safemasking,'Unit_clean'].str.replace(r'[A-Za-z]','',regex=True)
df['Unit_final']=pd.to_numeric(df.loc[safemasking,'Unit_clean'],errors='coerce')
df['Unit_final'].isna()
df['Unit_final'].notna()
df['Unit_final'].describe()
cleaning_report = {
    "rows_processed": len(df),
    "maximum":df['Unit_final'].max(),
    "minimum":df['Unit_final'].min(),
    "semanticflagging":df.loc[safemasking,'Unit_clean']
}

print(cleaning_report)
df[["Unit_final","Unit_raw"]].to_csv("cleaned_units.csv", index=False)


{'rows_processed': 20, 'maximum': np.float64(12000.0), 'minimum': np.float64(-12000.0), 'semanticflagging': 0          12
1         850
2          45
3           1
4            
6         780
7          15
8       ~3000
9       12000
10     999.99
11           
12       4500
13    1.20+04
14     -12000
15       7000
16         11
17        850
19       4500
Name: Unit_clean, dtype: object}


Amount

In [5]:
df['Total_Amount_raw']=df['Total_Amount']
df['Total_Amount_clean']=df['Total_Amount_raw'].astype(str).str.strip()
df[['Total_Amount_raw','Total_Amount_clean']]

,Total_Amount_raw,Total_Amount_clean
0,12000,12000
1,850,850
2,90,90
3,"1,100","1,100"
4,NaN,nan
5,infinite,infinite
6,1560,1560
7,1,1
8,3000,3000
9,12000,12000


In [6]:
df['missingval']=df['Total_Amount_clean'].str.match(r'\s*(n?a|na|missing|N/A|none|'')\s*$','',na=False)
df['infinityamt']=df['Total_Amount_raw'].str.match(r'\binf(inite)|inity|saturation|overflow|outofrange\b','',na=False)
df['Approximateamt']=df['Total_Amount_raw'].str.contains(r'\b(~|approximate|about|almost|nearly|approx|around)\b',case=False,na=False)
df['Isrange']=df['Total_Amount_raw'].str.contains(r'±|\+-?-',case=False,na=False,regex=True)
df['isinequalityamt']=df['Total_Amount_raw'].str.contains(r'<|>|<=|>=|\bless than|\bgreater than|\bbelow|\babove',na=False,case=False)
df['is_parenthesis']=df['Total_Amount_raw'].str.contains(r'\(\s*-?\d',case=False,na=False,regex=True)
safenet=~(df['missingval']|df['infinityamt']|df['Approximateamt']|df['Isrange']|df['isinequalityamt']|df['is_parenthesis'])
df.loc[safenet,'Total_Amount_raw']

C:\Users\ayush\AppData\Local\Temp\ipykernel_14320\2646970698.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['Approximateamt']=df['Total_Amount_raw'].str.contains(r'\b(~|approximate|about|almost|nearly|approx|around)\b',case=False,na=False)


0      12000
1        850
2         90
3      1,100
4        NaN
6       1560
7          1
8       3000
9      12000
10    999.99
11     13000
12      9000
13     12000
14    -12000
15      7000
16     11999
17      1700
18     12000
19      4500
Name: Total_Amount_raw, dtype: object

In [69]:
df.loc[safenet,'Total_Amount_clean']=df.loc[safenet,'Total_Amount_clean'].str.strip()
df.loc[safenet,'Total_Amount_clean']=df.loc[safenet,'Total_Amount_clean'].str.replace(r'[$₹]','',regex=True)
df.loc[safenet,'Total_Amount_clean']=df.loc[safenet,'Total_Amount_clean'].str.replace(r',','',regex=True)
df.loc[safenet,'Total_Amount_clean']=df.loc[safenet,'Total_Amount_clean'].str.replace(r'\b(usd|inr|rupees|RUPEES|rs|DOLLAR|INR)\b','',regex=True)
df['Amount_float']=pd.to_numeric(df.loc[safenet,'Total_Amount_clean'],errors='coerce')
df['Amount_float']
reportsummary={
    'Maximum':df['Amount_float'].max(),
    'Minimum':df['Amount_float'].min(),
    'mean':df['Amount_float'].mean(),
    'No of values':len(df['Amount_float'])}
df[['Total_Amount_clean','Amount_float']].to_csv("cleaned_numeric.csv", index=False)

Automate Numeric function

In [ ]:
def clean_numericcolumn (series):
    raw=series.copy()
    s=s.series.astype(str).str.strip().str.lower()
    s_raw=df['Total_Amount_raw'].astype(str)
    df['isinfinty']=s_raw.str.contains(r'\binf(inite)|overflow|saturation|outofrange\b','',regex=True)
    df['ismissing']=s_raw.str.match(r'^\s*(?:n/?a|missing|na|n\,a\.?|missing|none)?\s*$',case=False,na=False)
    df['isapprox']=s_raw.str.contains(r'~|±|approximate|around|about|approximately',case=False,na=False)
    df['ishyphen']=s_raw.str.contains(r'\(\s*-?\d',case=False,na=False)
    df['isrange']=s_raw.str.contains(r'\bless than|\bgreater than|\bbetween|>|<|>=|<=|\babove|\bbelow',na=False,case=False)
    df['amount_unit']=s_raw.str.contains(r'\b(k|K|million|thousand|m|b|billion)\b',case=False,na=False)
    df['isvariable']=s_raw.str.contains(r'+|\±/|-',na=False,case=False)
    s=s.str.lower().str.strip()
    s=s.str.replace(r'[$₹]','',regex=True)
    s=s.str.replace(r'\b(usd|inr|rupees|RS|DOLLAR|dollar?)\b','',regex=True,case=False)
    s=s.str.replace(r',','',regex=True)
    s=s.str.replace(r'\b(k|m|thousand|million)\b','',regex=True,case=False)
    s=s.str.replace(r'[~]','',regex=True)
    amount_numeric=pd.to_numeric(s.str.replace(r'[^\d\.]','',regex=True),errors='coerce')
    amount_numeric[df['flag_unitk']]*=1000
    amount_numeric[df['flag_unitm']]*=1000000
    df['Amount_numeric']=amount_numeric
    df[['Total_Amount_raw','Amount_numeric']] 

Date Column

In [37]:
# # semantic flagging
df['Order_raw']=df['Order_Date']
df['Date_clean']=df['Order_raw'].astype(str).str.strip()
df['hastextmonth']=df['Date_clean'].astype(str).str.contains(r'[A-Za-z]',na=False)
df['datemissing']=df['Date_clean'].astype(str).str.match(r'^\s*(?:na|n/?a|nan|none|missing)?\s*$',na=False)
df['date_hastime']=df['Date_clean'].astype(str).str.contains(r'\d{1,2}:\d{2}',na=False)

In [65]:

df['Date_clean']=df['Date_clean'].astype(str).str.strip()
df['Date_clean']=df['Date_clean'].str.replace(r'\b(\d{1,2})(st|nd|rd|th)\b',r'\1',regex=True,case=False)
df['Date_clean']=df['Date_clean'].str.replace(r'\b\d{1,2}:\d{2}(?::\d{2})?\s*(?:AM|PM|am|pm)?\b','',regex=True).str.strip()
df['Date_clean']=df['Date_clean'].str.replace(r'\b(?:IST|GMT|UTC|CET|EDT)\b','',regex=True,case=False)
df['Date_clean']=df['Date_clean'].str.replace(r'[+-]\d{2}:?\d{2}','',regex=True).str.strip()
df['Date_clean'] = df['Date_clean'].str.replace(r'[-/\.]', ' ', regex=True)
df['Date_clean'] = df['Date_clean'].str.replace(r'\s+', ' ', regex=True).str.strip()


df.loc[df['hastextmonth'],'dateparsed'] = pd.to_datetime(
    df.loc[df['hastextmonth'],'Date_clean'], errors='coerce', dayfirst=True
)


df['is_numeric_date'] = (~df['hastextmonth'] & df['Date_clean'].astype(str).str.match(r'^\d'))
df.loc[df['is_numeric_date'],'dateparsed'] = pd.to_datetime(
    df.loc[df['is_numeric_date'],'Date_clean'], errors='coerce', dayfirst=True
)

df['dat_parsed_failed']=(df['is_numeric_date']&df['dateparsed'].isna())
def twodigit(x):
    x=str(x)
    parts=x.split()
    if len(parts)!=3:
        return x
    d,m,y=parts
    if len(y)==2 and y.isdigit():
        y=int(y)
        y=2000+y if y<=30 else 1900+y
    return f'{d} {m} {y}'

df.loc[df['dat_parsed_failed'],'Date_clean'] = df.loc[df['dat_parsed_failed'],'Date_clean'].apply(twodigit)
df.loc[df['dat_parsed_failed'],'dateparsed'] = pd.to_datetime(
    df.loc[df['dat_parsed_failed'],'Date_clean'], dayfirst=True, errors='coerce'
)
df['dateparsed']


0     2025-03-02 00:00:00
1     2025-04-03 00:00:00
2     2025-04-05 00:00:00
3     2025-05-04 00:00:00
4     2025-05-06 00:00:00
5     2025-07-05 00:00:00
6     2025-01-08 00:00:00
7                     NaT
8     2025-01-08 00:00:00
9                     NaT
10                    NaT
11    2025-11-08 00:00:00
12    2025-08-11 00:00:00
13                    NaT
14    2025-08-12 00:00:00
15    2025-08-13 00:00:00
16                    NaT
17    2025-08-15 00:00:00
18                    NaT
19                    NaT
Name: dateparsed, dtype: object

In [71]:
df[["Date_clean","dateparsed"]].to_csv("cleaned_dates.csv", index=False)
